# Agent Loan Decisioning

In [63]:
import sys
import datetime
from datetime import date,timedelta
import numpy as np
import pandas as pd
# import camelot
import matplotlib.pyplot as plt
import logging

In [64]:
pd.set_option('display.max_columns',None)

In [65]:
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
logging.info('Code Begin')

2023-05-18 17:27:50,513 - Code Begin


In [66]:
# sys.maxsize
# 2147483647



Qualifying Matrix

1.	Agent Length of Service - 3 months

2.	Agent Frequency of use/activity - >= 65 transactions(count) in total

3.	Agent Frequency of rebalance - >= 45 inbound(count) in total

4.	Median Cash-In for last 90 days - avg. txn amnt - ghs5000 

5.	Mode Cash-In for last 90 days - lowest amnt cash in - ghs1000 

6.	Median Cash-Out for last 90 days - avg. txn amnt - ghs5000

7.	Mode Cash-Out for last 90 days - lowest amnt cash out - ghs1000 

In [67]:

# ##### full pdf reading

# # tables = camelot.read_pdf(r'C:\Users\Chakdahah\MomoStatementReport.pdf',pages='all') #emma data
# # tables = camelot.read_pdf(r'C:\Users\Chakdahah\momopages2.pdf',pages='all') # 2 pages prince
# # tables = camelot.read_pdf(r'C:\Users\Chakdahah\Momopages3.pdf',pages='all') #50 pages prince
# # tables = camelot.read_pdf(r'C:\Users\Chakdahah\lastlast.pdf',pages='all') # 5 pages prince containig each month 
# # tables = camelot.read_pdf(r'C:\Users\Chakdahah\MomoStatement98.pdf',pages='all') # 98 pages prince containig each month 26mi
# # tables = camelot.read_pdf(r'C:\Users\Chakdahah\MomoStatementmax.pdf',pages='all') # maxwell 235 pages 
# # tables = camelot.read_pdf(r'C:\Users\Chakdahah\MomoStatement256.pdf',pages='all') # maxwell 256 pages 
# tables = camelot.read_pdf(r'C:\Users\Chakdahah\MomoStatement298.pdf',pages='all') # amokoaa regina 298 pages  

# tables



# # tables[j].parsing_report #check parsing report to see accuracy of parsing and other details

# # append to one csv file

# table_fin = tables[0].df #convert first table to df and assign to variable 

# for i in range(1,len(tables)-1): # skip 0-nth of tables
#     print(i)
#     t = tables[i].df 
#     t.drop(t.index[0], inplace = True) # drop first row of df
#     table_fin = pd.concat([table_fin, t], axis = 0)
    

#     print(t)


In [68]:
table_fin = pd.read_csv('/Users/otema/benjamin_adjei_statement.csv')
table_fin.head()

,Id,External Transaction Id,Date,Status,Type,Provider Category,To Message,From,From Name,To,To Name,Initiated By,On Behalf Of,Approved By,Amount,Currency,Fee,Currency.1,Discount,Currency.2,Promotion,Currency.3,Coupon,Currency.4,Balance,Currency.5,Information
0,23515712900,NaN,4/11/23 11:51,Successful,Cash in,NaN,Cashin To-233598355319,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233598355319/MSISDN,JOSEPH KWAME FRIMPONG KESSE,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,-20,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,56,GHS,NaN
1,23515677808,NaN,4/11/23 11:50,Successful,Cash in,NaN,Cashin To-233557245702,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233557245702/MSISDN,Linda Opoku,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,-70,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,77,GHS,NaN
2,23515648162,NaN,4/11/23 11:48,Successful,Cash in,NaN,Cashin To-233547877892,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233547877892/MSISDN,Emmanuel Ahorgah,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,-15,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,147,GHS,NaN
3,23515519054,NaN,4/11/23 11:44,Successful,Cash out,NaN,NationalId--,FRI:233543859094/MSISDN,DERRICK AHIAGBENYO,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233543859094/MSISDN,ID:233543859094/MSISDN,NaN,30,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,162,GHS,NaN
4,23515477521,NaN,4/11/23 11:42,Successful,Cash out,NaN,NationalId--,FRI:233558384831/MSISDN,Patience Zionu,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233558384831/MSISDN,ID:233558384831/MSISDN,NaN,50,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,132,GHS,NaN


In [69]:
# Assign tables into momo data variable

momo_data = table_fin
momo_data

,Id,External Transaction Id,Date,Status,Type,Provider Category,To Message,From,From Name,To,To Name,Initiated By,On Behalf Of,Approved By,Amount,Currency,Fee,Currency.1,Discount,Currency.2,Promotion,Currency.3,Coupon,Currency.4,Balance,Currency.5,Information
0,23515712900,NaN,4/11/23 11:51,Successful,Cash in,NaN,Cashin To-233598355319,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233598355319/MSISDN,JOSEPH KWAME FRIMPONG KESSE,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,-20,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,56,GHS,NaN
1,23515677808,NaN,4/11/23 11:50,Successful,Cash in,NaN,Cashin To-233557245702,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233557245702/MSISDN,Linda Opoku,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,-70,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,77,GHS,NaN
2,23515648162,NaN,4/11/23 11:48,Successful,Cash in,NaN,Cashin To-233547877892,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233547877892/MSISDN,Emmanuel Ahorgah,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,-15,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,147,GHS,NaN
3,23515519054,NaN,4/11/23 11:44,Successful,Cash out,NaN,NationalId--,FRI:233543859094/MSISDN,DERRICK AHIAGBENYO,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233543859094/MSISDN,ID:233543859094/MSISDN,NaN,30,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,162,GHS,NaN
4,23515477521,NaN,4/11/23 11:42,Successful,Cash out,NaN,NationalId--,FRI:233558384831/MSISDN,Patience Zionu,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233558384831/MSISDN,ID:233558384831/MSISDN,NaN,50,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,132,GHS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6754,21823172071,NaN,2/1/23 7:01,Successful,Cash out,NaN,NationalId--,FRI:233595281495/MSISDN,BEATRICE AKPALIGAH,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233595281495/MSISDN,ID:233595281495/MSISDN,NaN,30,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,251,GHS,NaN
6755,21823144046,230201MMB2187873,2/1/23 6:59,Successful,External payment,NaN,NaN,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:1441002407704@ecobankpush/SP,Ecobank,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,-2000,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,221,GHS,NaN
6756,21822953068,NaN,2/1/23 6:41,Successful,Cash out,NaN,NationalId--,FRI:233248527306/MSISDN,Nana Nyarko,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233248527306/MSISDN,ID:233248527306/MSISDN,NaN,500,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,2221,GHS,NaN
6757,21822942874,NaN,2/1/23 6:40,Successful,Cash out,NaN,NationalId--,FRI:233550878266/MSISDN,JENNIEFER AGBASAH,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233550878266/MSISDN,ID:233550878266/MSISDN,NaN,20,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,1721,GHS,NaN


In [70]:
# momo_data[~momo_data.iloc[:,0].str.contains("\n")]

In [71]:
# momo_data.columns = momo_data.iloc[0] #set column names equal to values in row index position 0
# momo_data = momo_data[1:] # remove first row from DataFrame

In [72]:
# momo_data.reset_index(drop=True, inplace=True)

In [73]:
# momo_data

In [74]:
#Reading data from csv file

# momo_data = pd.read_csv(r'\Users\Chakdahah\Cleaned_Momo.csv', skipinitialspace = True)

# Clean whitespaces in colunn name and entire dataframe

momo_data =momo_data.rename(columns=lambda x: x.strip())
momo_data =momo_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

momo_data.head()

,Id,External Transaction Id,Date,Status,Type,Provider Category,To Message,From,From Name,To,To Name,Initiated By,On Behalf Of,Approved By,Amount,Currency,Fee,Currency.1,Discount,Currency.2,Promotion,Currency.3,Coupon,Currency.4,Balance,Currency.5,Information
0,23515712900,NaN,4/11/23 11:51,Successful,Cash in,NaN,Cashin To-233598355319,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233598355319/MSISDN,JOSEPH KWAME FRIMPONG KESSE,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,-20,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,56,GHS,NaN
1,23515677808,NaN,4/11/23 11:50,Successful,Cash in,NaN,Cashin To-233557245702,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233557245702/MSISDN,Linda Opoku,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,-70,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,77,GHS,NaN
2,23515648162,NaN,4/11/23 11:48,Successful,Cash in,NaN,Cashin To-233547877892,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233547877892/MSISDN,Emmanuel Ahorgah,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,-15,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,147,GHS,NaN
3,23515519054,NaN,4/11/23 11:44,Successful,Cash out,NaN,NationalId--,FRI:233543859094/MSISDN,DERRICK AHIAGBENYO,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233543859094/MSISDN,ID:233543859094/MSISDN,NaN,30,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,162,GHS,NaN
4,23515477521,NaN,4/11/23 11:42,Successful,Cash out,NaN,NationalId--,FRI:233558384831/MSISDN,Patience Zionu,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233558384831/MSISDN,ID:233558384831/MSISDN,NaN,50,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,132,GHS,NaN


In [75]:

# remove unwanted characters from end of column names
# mmomo_data.rename(columns=lambda x: x.strip('.'),inplace =True)

# remove all unwanted char acters in column names

momo_data.rename(columns=lambda s: s.replace(".", ""), inplace=True)

# replace whitespaces with underscore
momo_data.rename(columns=lambda s: s.replace(" ", "_"), inplace=True)

In [77]:
# rename column names here

momo_data.rename(
    columns = {'From':"FROM_NO", "From_Name":"FROM_NAME","Type":"TRANS_TYPE",
              "Amount":"AMOUNT","Initiated_By":"OVA","Date":"TRANSACTION_DATE"},
                inplace = True)
    
momo_data

,Id,External_Transaction_Id,TRANSACTION_DATE,Status,TRANS_TYPE,Provider_Category,To_Message,FROM_NO,FROM_NAME,To,To_Name,OVA,On_Behalf_Of,Approved_By,AMOUNT,Currency,Fee,Currency1,Discount,Currency2,Promotion,Currency3,Coupon,Currency4,Balance,Currency5,Information
0,23515712900,NaN,4/11/23 11:51,Successful,Cash in,NaN,Cashin To-233598355319,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233598355319/MSISDN,JOSEPH KWAME FRIMPONG KESSE,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,-20,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,56,GHS,NaN
1,23515677808,NaN,4/11/23 11:50,Successful,Cash in,NaN,Cashin To-233557245702,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233557245702/MSISDN,Linda Opoku,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,-70,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,77,GHS,NaN
2,23515648162,NaN,4/11/23 11:48,Successful,Cash in,NaN,Cashin To-233547877892,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233547877892/MSISDN,Emmanuel Ahorgah,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,-15,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,147,GHS,NaN
3,23515519054,NaN,4/11/23 11:44,Successful,Cash out,NaN,NationalId--,FRI:233543859094/MSISDN,DERRICK AHIAGBENYO,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233543859094/MSISDN,ID:233543859094/MSISDN,NaN,30,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,162,GHS,NaN
4,23515477521,NaN,4/11/23 11:42,Successful,Cash out,NaN,NationalId--,FRI:233558384831/MSISDN,Patience Zionu,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233558384831/MSISDN,ID:233558384831/MSISDN,NaN,50,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,132,GHS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6754,21823172071,NaN,2/1/23 7:01,Successful,Cash out,NaN,NationalId--,FRI:233595281495/MSISDN,BEATRICE AKPALIGAH,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233595281495/MSISDN,ID:233595281495/MSISDN,NaN,30,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,251,GHS,NaN
6755,21823144046,230201MMB2187873,2/1/23 6:59,Successful,External payment,NaN,NaN,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:1441002407704@ecobankpush/SP,Ecobank,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,-2000,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,221,GHS,NaN
6756,21822953068,NaN,2/1/23 6:41,Successful,Cash out,NaN,NationalId--,FRI:233248527306/MSISDN,Nana Nyarko,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233248527306/MSISDN,ID:233248527306/MSISDN,NaN,500,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,2221,GHS,NaN
6757,21822942874,NaN,2/1/23 6:40,Successful,Cash out,NaN,NationalId--,FRI:233550878266/MSISDN,JENNIEFER AGBASAH,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233550878266/MSISDN,ID:233550878266/MSISDN,NaN,20,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,1721,GHS,NaN


In [78]:
momo_data["AMOUNT"] = momo_data["AMOUNT"].astype(str)

In [79]:
# remove negative sign in amount column

momo_data["AMOUNT"] = momo_data["AMOUNT"].str.replace("-","")

In [80]:
momo_data["TRANS_TYPE"] = momo_data["TRANS_TYPE"].str.upper()
momo_data["TRANS_TYPE"] = momo_data["TRANS_TYPE"].str.replace(" ","_")


In [81]:
momo_data

,Id,External_Transaction_Id,TRANSACTION_DATE,Status,TRANS_TYPE,Provider_Category,To_Message,FROM_NO,FROM_NAME,To,To_Name,OVA,On_Behalf_Of,Approved_By,AMOUNT,Currency,Fee,Currency1,Discount,Currency2,Promotion,Currency3,Coupon,Currency4,Balance,Currency5,Information
0,23515712900,NaN,4/11/23 11:51,Successful,CASH_IN,NaN,Cashin To-233598355319,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233598355319/MSISDN,JOSEPH KWAME FRIMPONG KESSE,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,20.5,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,56,GHS,NaN
1,23515677808,NaN,4/11/23 11:50,Successful,CASH_IN,NaN,Cashin To-233557245702,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233557245702/MSISDN,Linda Opoku,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,70.0,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,77,GHS,NaN
2,23515648162,NaN,4/11/23 11:48,Successful,CASH_IN,NaN,Cashin To-233547877892,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233547877892/MSISDN,Emmanuel Ahorgah,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,15.0,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,147,GHS,NaN
3,23515519054,NaN,4/11/23 11:44,Successful,CASH_OUT,NaN,NationalId--,FRI:233543859094/MSISDN,DERRICK AHIAGBENYO,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233543859094/MSISDN,ID:233543859094/MSISDN,NaN,30.0,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,162,GHS,NaN
4,23515477521,NaN,4/11/23 11:42,Successful,CASH_OUT,NaN,NationalId--,FRI:233558384831/MSISDN,Patience Zionu,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233558384831/MSISDN,ID:233558384831/MSISDN,NaN,50.0,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,132,GHS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6754,21823172071,NaN,2/1/23 7:01,Successful,CASH_OUT,NaN,NationalId--,FRI:233595281495/MSISDN,BEATRICE AKPALIGAH,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233595281495/MSISDN,ID:233595281495/MSISDN,NaN,30.0,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,251,GHS,NaN
6755,21823144046,230201MMB2187873,2/1/23 6:59,Successful,EXTERNAL_PAYMENT,NaN,NaN,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:1441002407704@ecobankpush/SP,Ecobank,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,2000.0,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,221,GHS,NaN
6756,21822953068,NaN,2/1/23 6:41,Successful,CASH_OUT,NaN,NationalId--,FRI:233248527306/MSISDN,Nana Nyarko,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233248527306/MSISDN,ID:233248527306/MSISDN,NaN,500.0,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,2221,GHS,NaN
6757,21822942874,NaN,2/1/23 6:40,Successful,CASH_OUT,NaN,NationalId--,FRI:233550878266/MSISDN,JENNIEFER AGBASAH,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233550878266/MSISDN,ID:233550878266/MSISDN,NaN,20.0,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,1721,GHS,NaN


In [82]:
momo = momo_data.loc[momo_data['OVA'].str.contains("cis", case = False)]
momo

,Id,External_Transaction_Id,TRANSACTION_DATE,Status,TRANS_TYPE,Provider_Category,To_Message,FROM_NO,FROM_NAME,To,To_Name,OVA,On_Behalf_Of,Approved_By,AMOUNT,Currency,Fee,Currency1,Discount,Currency2,Promotion,Currency3,Coupon,Currency4,Balance,Currency5,Information
319,23452549186,7.09E+17,4/8/23 20:19,Successful,DEBIT,NaN,171-Internet-Data_Flexi_Bundle,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:cis/USER,MTNBUNDLE .,ID:cis/USER,ID:cis/USER,NaN,3.0,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,954,GHS,NaN
602,23402883542,8.55E+17,4/7/23 7:42,Successful,DEBIT,NaN,138-Internet-Data_Flexi_Bundle,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:cisnew/USER,MTN,ID:cisnew/USER,ID:cisnew/USER,NaN,3.0,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,1110,GHS,NaN
1232,23231390589,9.97E+17,4/1/23 8:31,Successful,DEBIT,NaN,138-Internet-Data_Flexi_Bundle,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:cisnew/USER,MTN,ID:cisnew/USER,ID:cisnew/USER,NaN,4.5,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,778,GHS,NaN
2075,23056526497,3.27E+17,3/25/23 20:03,Successful,DEBIT,NaN,138-Internet-Data_Flexi_Bundle,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:cisnew/USER,MTN,ID:cisnew/USER,ID:cisnew/USER,NaN,0.9,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,401,GHS,NaN


In [83]:
# cast appropriate data types
momo_data['AMOUNT'] = pd.to_numeric(momo_data['AMOUNT'],errors='coerce')
# momo_data['BAL_BEFORE'] = pd.to_numeric(momo_data['BAL_BEFORE'],errors='coerce')
# momo_data['BAL_AFTER'] = pd.to_numeric(momo_data['BAL_AFTER'],errors='coerce')
# momo_data['F_ID'] = pd.to_numeric(momo_data['F_ID'],errors='coerce')


In [84]:
momo_data.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6759 entries, 0 to 6758
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Id                       6759 non-null   int64  
 1   External_Transaction_Id  822 non-null    object 
 2   TRANSACTION_DATE         6759 non-null   object 
 3   Status                   6759 non-null   object 
 4   TRANS_TYPE               6759 non-null   object 
 5   Provider_Category        0 non-null      float64
 6   To_Message               5967 non-null   object 
 7   FROM_NO                  6759 non-null   object 
 8   FROM_NAME                6759 non-null   object 
 9   To                       6759 non-null   object 
 10  To_Name                  6759 non-null   object 
 11  OVA                      6759 non-null   object 
 12  On_Behalf_Of             6759 non-null   object 
 13  Approved_By              4 non-null      object 
 14  AMOUNT                  

In [85]:
#Suppressing any scientific notation in entire csv file.

pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [86]:
momo_data.head()

,Id,External_Transaction_Id,TRANSACTION_DATE,Status,TRANS_TYPE,Provider_Category,To_Message,FROM_NO,FROM_NAME,To,To_Name,OVA,On_Behalf_Of,Approved_By,AMOUNT,Currency,Fee,Currency1,Discount,Currency2,Promotion,Currency3,Coupon,Currency4,Balance,Currency5,Information
0,23515712900,NaN,4/11/23 11:51,Successful,CASH_IN,NaN,Cashin To-233598355319,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233598355319/MSISDN,JOSEPH KWAME FRIMPONG KESSE,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,20,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,56,GHS,NaN
1,23515677808,NaN,4/11/23 11:50,Successful,CASH_IN,NaN,Cashin To-233557245702,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233557245702/MSISDN,Linda Opoku,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,70,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,77,GHS,NaN
2,23515648162,NaN,4/11/23 11:48,Successful,CASH_IN,NaN,Cashin To-233547877892,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:233547877892/MSISDN,Emmanuel Ahorgah,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,15,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,147,GHS,NaN
3,23515519054,NaN,4/11/23 11:44,Successful,CASH_OUT,NaN,NationalId--,FRI:233543859094/MSISDN,DERRICK AHIAGBENYO,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233543859094/MSISDN,ID:233543859094/MSISDN,NaN,30,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,162,GHS,NaN
4,23515477521,NaN,4/11/23 11:42,Successful,CASH_OUT,NaN,NationalId--,FRI:233558384831/MSISDN,Patience Zionu,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233558384831/MSISDN,ID:233558384831/MSISDN,NaN,50,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,132,GHS,NaN


In [87]:
m =momo_data['TRANSACTION_DATE'].str.split(' ', expand=True)


In [88]:

momo_data['TRANSACTION_DATE'] = m[0]
momo_data['TRANSACTION_TIME'] = m[1]


In [89]:
# shift column 'C' to first position
first_column = momo_data.pop('TRANSACTION_TIME')
  
# insert column using insert(position,column_name,first_column) function
momo_data.insert(1, 'TRANSACTION_TIME', first_column)
  

In [90]:
momo_data.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6759 entries, 0 to 6758
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Id                       6759 non-null   int64  
 1   TRANSACTION_TIME         6759 non-null   object 
 2   External_Transaction_Id  822 non-null    object 
 3   TRANSACTION_DATE         6759 non-null   object 
 4   Status                   6759 non-null   object 
 5   TRANS_TYPE               6759 non-null   object 
 6   Provider_Category        0 non-null      float64
 7   To_Message               5967 non-null   object 
 8   FROM_NO                  6759 non-null   object 
 9   FROM_NAME                6759 non-null   object 
 10  To                       6759 non-null   object 
 11  To_Name                  6759 non-null   object 
 12  OVA                      6759 non-null   object 
 13  On_Behalf_Of             6759 non-null   object 
 14  Approved_By             

In [91]:
# momo_data['FROM_NAME'] = momo_data.FROM_NAME.str.replace('\n',' ')

In [92]:
momo_data.tail()

,Id,TRANSACTION_TIME,External_Transaction_Id,TRANSACTION_DATE,Status,TRANS_TYPE,Provider_Category,To_Message,FROM_NO,FROM_NAME,To,To_Name,OVA,On_Behalf_Of,Approved_By,AMOUNT,Currency,Fee,Currency1,Discount,Currency2,Promotion,Currency3,Coupon,Currency4,Balance,Currency5,Information
6754,21823172071,7:01,NaN,2/1/23,Successful,CASH_OUT,NaN,NationalId--,FRI:233595281495/MSISDN,BEATRICE AKPALIGAH,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233595281495/MSISDN,ID:233595281495/MSISDN,NaN,30,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,251,GHS,NaN
6755,21823144046,6:59,230201MMB2187873,2/1/23,Successful,EXTERNAL_PAYMENT,NaN,NaN,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:1441002407704@ecobankpush/SP,Ecobank,ID:233248341190/MSISDN,ID:233248341190/MSISDN,NaN,2000,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,221,GHS,NaN
6756,21822953068,6:41,NaN,2/1/23,Successful,CASH_OUT,NaN,NationalId--,FRI:233248527306/MSISDN,Nana Nyarko,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233248527306/MSISDN,ID:233248527306/MSISDN,NaN,500,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,2221,GHS,NaN
6757,21822942874,6:40,NaN,2/1/23,Successful,CASH_OUT,NaN,NationalId--,FRI:233550878266/MSISDN,JENNIEFER AGBASAH,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233550878266/MSISDN,ID:233550878266/MSISDN,NaN,20,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,1721,GHS,NaN
6758,21822900312,6:35,NaN,2/1/23,Successful,CASH_OUT,NaN,NationalId--,FRI:233243442321/MSISDN,Theophilus Frimpong,FRI:233248341190/MSISDN,BENJAMIN ADJEI,ID:233243442321/MSISDN,ID:233243442321/MSISDN,NaN,1700,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,1701,GHS,NaN


In [102]:
momo_data['FROM_NO'] = momo_data['FROM_NO'].apply(lambda st: st[st.find(":")+1:st.find("/")])
momo_data['FROM_NO'] = momo_data['FROM_NO'].apply(lambda st: st[st.find(":")+1:st.find("@")])


In [103]:
num=momo_data.loc[momo_data['TRANS_TYPE'] == 'CASH_IN', 'FROM_NO'].iloc[0]
num

'2332483411'

In [94]:
momo = momo_data.loc[momo_data['OVA'].str.contains("cis", case = False)]
momo

,Id,TRANSACTION_TIME,External_Transaction_Id,TRANSACTION_DATE,Status,TRANS_TYPE,Provider_Category,To_Message,FROM_NO,FROM_NAME,To,To_Name,OVA,On_Behalf_Of,Approved_By,AMOUNT,Currency,Fee,Currency1,Discount,Currency2,Promotion,Currency3,Coupon,Currency4,Balance,Currency5,Information
319,23452549186,20:19,7.09E+17,4/8/23,Successful,DEBIT,NaN,171-Internet-Data_Flexi_Bundle,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:cis/USER,MTNBUNDLE .,ID:cis/USER,ID:cis/USER,NaN,3,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,954,GHS,NaN
602,23402883542,7:42,8.55E+17,4/7/23,Successful,DEBIT,NaN,138-Internet-Data_Flexi_Bundle,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:cisnew/USER,MTN,ID:cisnew/USER,ID:cisnew/USER,NaN,3,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,1110,GHS,NaN
1232,23231390589,8:31,9.97E+17,4/1/23,Successful,DEBIT,NaN,138-Internet-Data_Flexi_Bundle,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:cisnew/USER,MTN,ID:cisnew/USER,ID:cisnew/USER,NaN,4,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,778,GHS,NaN
2075,23056526497,20:03,3.27E+17,3/25/23,Successful,DEBIT,NaN,138-Internet-Data_Flexi_Bundle,FRI:233248341190/MSISDN,BENJAMIN ADJEI,FRI:cisnew/USER,MTN,ID:cisnew/USER,ID:cisnew/USER,NaN,1,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,401,GHS,NaN


In [95]:
name = momo['FROM_NAME'].iloc[0]
name

'BENJAMIN ADJEI'

In [ ]:
# name=momo_data.loc[momo_data['TRANS_TYPE'] == 'CASH_IN', 'FROM_NAME'].iloc[0]
# name

In [ ]:
momo_data[~momo_data.iloc[:,0].str.contains("\n")]

In [ ]:
df3=momo_data.loc[momo_data['TRANS_TYPE'] == 'CASH_IN', 'FROM_NAME'].iloc[0]
df3

In [ ]:
momo_data =momo_data.dropna()

In [ ]:
# store date as object for later use
mi = momo_data['TRANSACTION_DATE']

In [ ]:
momo_data['TRANSACTION_DATE'] = pd.to_datetime(momo_data['TRANSACTION_DATE'])

# Converting PDF to CSV and XLSX


In [ ]:
## convert momo_data to csv
momo_data.to_csv(r'C:\Users\Chakdahah\Downloads\Django-main\Django-main\analytics_project\StrippedMomotoCSV298.csv',date_format='%d/%m/%Y')

momo_data.to_excel("StrippedMomotoXLS298.xlsx")

In [ ]:
# Set empty list and dict
masterData = []
userData= {}

# Calculation of scoring parameters

In [ ]:

rows = momo_data.count()[0] # Agent Frequency of use
userData["freq_of_active"] = rows



MOMO=momo_data[(momo_data == 'CASH_IN').any(axis=1)] # frequency of rebalance
MOMOR = MOMO.count()[0]
userData["freq_of_repl"] = MOMOR


meadnie = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_IN','AMOUNT'].median() #Median Cash-In for last 90days
# userData["mead_cash_in"] = meadnie
# userData["mead_cash_in"]
userData["median_cash_in_calc"] = meadnie*4


modnie = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_IN','AMOUNT'].mode() # Mode Cash-In for last 90 days
userData["mode_cash_in_calc"] = modnie*4
userData["mode_cash_in_calc"] = max(userData["mode_cash_in_calc"]) #for converting from bool to int for processing


meadniout = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_OUT','AMOUNT'].median() # Median Cash-Out for last 90 days
# userData["mead_cash_out"] = meadniout
# userData["mead_cash_out"]
userData["median_cash_out_calc"] = meadniout*4


modno = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_OUT','AMOUNT'].mode() # Mode Cash-Out for last 90 days
userData["mode_cash_out_calc"] = modno*4 
userData["mode_cash_out_calc"] = max(userData["mode_cash_out_calc"])




In [ ]:
# momo_data.loc[momo_data['TRANS_TYPE']=='CASH_OUT','AMOUNT'].mode() # Mode Cash-Out for last 90 days


In [ ]:

# print(modnie)
# userData["mode_cash_in_calc"] = modnie*4
# print(userData["mode_cash_in_calc"])
# # userData["mode_cash_out_calc"] = userData["mode_cash_out_calc"]*4

# # print(max(userData["mode_cash_in_calc"]))

In [ ]:
# print(max(userData["mode_cash_in_calc"]))

# Scoring Criteria Calculation

In [ ]:
user_copy = userData.copy()
masterData.append(user_copy)

In [ ]:
### scoring

inelligible = []
elligible = []
score_table = []
scoring_table = {}
combined = []

In [ ]:
scoring_table["Agent Number"] = num
scoring_table["Agent Name"] = name

In [ ]:


## Agent Length of Service - 3 months

for i in mi:
    u = mi.str.extract('([a-zA-Z ]+)', expand=False).str.strip()
    monthcount = u.drop_duplicates()
print(monthcount)

if len(monthcount)>=3:
    scoring_table["Length of Service"] = 100
    Credit_score = 100
else:
    scoring_table["Length of Service"] = 0
    Credit_score = 0


for index in range(len(masterData)):

    ## Frequency of use  >= 65 transactions(count) in total

    if masterData[index]["freq_of_active"]> 65 and masterData[index]["freq_of_active"] <= 75:
        user_copy = masterData[index]
#         print(user_copy)
        Credit_score += 50
        scoring_table["Frequency of Activity"] = 50
    elif masterData[index]["freq_of_active"]> 75:
        elig_copy = masterData[index]
        Credit_score+= 100
        scoring_table["Frequency of Activity"] = 100
    else:
        scoring_table["Frequency of Activity"] = 0


        

        
        
#Agent Frequency of rebalance - >= 45 inbound(count) in total

    if masterData[index]["freq_of_repl"]>= 45 and masterData[index]["freq_of_repl"] <= 54:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Frequency of Rebalance"] = 50
    elif masterData[index]["freq_of_active"]> 54:
        elig_copy = masterData[index]
        Credit_score+= 100
        scoring_table["Frequency of Rebalance"] = 100
    else:
        scoring_table["Frequency of Rebalance"] = 0
        

        
        
        # Median Cash-In for last 90 days - avg. txn amnt - ghs5000 

        
    if masterData[index]["median_cash_in_calc"] >= 5000:
        user_copy = masterData[index]
        Credit_score += 50
        scoring_table["Median Cash In"] = 50
    else:
        Credit_score+= 0
        scoring_table["Median Cash In"] = 0

        
        
          # Mode Cash-In for last 90 days - lowest amnt cash in - ghs1000 

        
    if masterData[index]["mode_cash_in_calc"] >= 1000:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Mode Cash In"] = 50
        
    else:
        elig_copy = masterData[index]
        Credit_score+= 0
        scoring_table["Mode Cash In"] = 0
        
        
        
       # Median Cash-Out for last 90 days - avg. txn amnt - ghs5000


    if masterData[index]["median_cash_out_calc"] > 5000:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Median Cash Out"] = 50
    else:
        elig_copy = masterData[index]
        Credit_score+= 0
        scoring_table["Median Cash Out"] = 0
        
#     Mode Cash-Out for last 90 days - lowest amnt cash out - ghs1000 
        
    if masterData[index]["mode_cash_out_calc"] >= 1000:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Mode Cash Out"] = 50
    else:
        elig_copy = masterData[index]
        Credit_score+= 0   
        scoring_table["Mode Cash Out"] = 0
    

#         print(elig_copy)
scoring_table["Credit Score"] = Credit_score
score_table.append(scoring_table)        
        
    

In [ ]:
elligible.append(elig_copy)

# Applying scores to Merchants

In [ ]:
    
    if Credit_score >= 200 and Credit_score <= 250:
        print("You qualify for a GHS 1000 loan")
        scoring_table["Eligibility"] = 1000

    elif Credit_score >= 251 and Credit_score <= 350:
        print("You qualify for a GHS 2000 loan")
        scoring_table["Eligibility"] = 2000

    elif Credit_score >= 351 and Credit_score <= 400:
        print("You qualify for a GHS 3000 loan")
        scoring_table["Eligibility"] = 3000


    elif Credit_score >= 401 and Credit_score <= 500:
        print("You qualify for a GHS 5000 loan")
        scoring_table["Eligibility"] = 5000



# Combining Eligibility and Score Tables

In [ ]:
comb = scoring_table | elig_copy
comb


In [ ]:
combined.append(comb)

In [ ]:
print("Elligible", elligible)

In [ ]:
Credit_score

# Converting and saving Score Table

In [ ]:
score_table = pd.DataFrame.from_dict(score_table)
score_table

In [ ]:
score_table.to_csv(r'C:\Users\Chakdahah\Downloads\Django-main\Django-main\analytics_project\scoretable298.csv')

# Converting and saving Eligibility Table

In [ ]:
### convert dict to df

elligible = pd.DataFrame.from_dict(elligible)
elligible

In [ ]:
type(elligible)

In [ ]:
elligible.to_csv(r'C:\Users\Chakdahah\Downloads\Django-main\Django-main\analytics_project\elligible298.csv')

# Converting and saving Combined Table

In [ ]:
combined_table = pd.DataFrame.from_dict(combined)
combined_table

In [ ]:
combined_table.to_csv(r'C:\Users\Chakdahah\Downloads\Django-main\Django-main\analytics_project\combtable298.csv')

In [ ]:
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
logging.info('End of Code')

# Score and risk grade table 

In [ ]:
details = {
    'Risk Ranking' : ['RR-1', 'RR-2', 'RR-3', 'RR-4'],
    'Minimum Score' : [200,251,351,401],
    'Maximum Score' : [250,350,400,500],
    'Qualifying Amount(GHS)':[1000,2000,3000,5000]
}
  
# creating a Dataframe object 
df = pd.DataFrame(details)
df